In [172]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
#warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import hamming_loss
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.cluster import KMeans
from operator import itemgetter
from imblearn.over_sampling import SMOTE

### Solution to 1(a) as below:

We've downloaded the dataset below and used sklearn.model_selection.train_test_split to split choose 70% of the data randomly as training data. 

In [173]:
fcalls=pd.read_csv('/Users/sharadsharma/Downloads/Anuran/Frogs_MFCCs.csv')
del fcalls['RecordID']

In [174]:
fcalls.shape

(7195, 25)

In [175]:
fcalls.head(5)

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre


In [176]:
Y_fcalls=fcalls.iloc[:,[-3,-2,-1]]
X_fcalls=fcalls.iloc[:,:-3]
X_tr, X_ts, Y_tr, Y_ts = train_test_split(X_fcalls, Y_fcalls, test_size = 0.3, random_state = 0)

In [177]:
X_tr.shape

(5036, 22)

In [178]:
Y_tr.shape

(5036, 3)

In [179]:
X_ts.shape

(2159, 22)

In [180]:
Y_ts.shape

(2159, 3)

### Solution to 1(b)(i) as below:

We're going to use exact match and hamming loss methods for evaluating the classifiers in our problem set.

Exact Match : It is a very strict metric. It tells us what percentage of the samples had all their labels classified correctly. This can be unfair to classifiers which, for eg, predict 2 out of three 3 labels correctly. It ignores partially correct matches.

Hamming Loss : It is not as strict as exact match and rewards for partial-correctness as well. It is fraction of lables incorrectly predicted to the total number of labels.

### Solution to 1(b)(ii) as below:

We'll first try and find the upper and lower bounds for the weight of the SVM penalty and the width of
the Gaussian Kernel. We'll do this for all the labels.

After that,for each label, we'll use Cross Validation on the chosen values on C and gamma to find our best pair of (C,gamma). We'll then train our data on these chosen parameters and find the hamming loss and exact match for each label.

Towards the end, we compute the hamming score and the exact match score for the classifier as a whole as well.

In [181]:
Y_tr_family=Y_tr[['Family']]
Y_tr_genus= Y_tr[['Genus']]
Y_tr_species= Y_tr[['Species']]

Y_ts_family=Y_ts[['Family']]
Y_ts_genus= Y_ts[['Genus']]
Y_ts_species= Y_ts[['Species']]


#### We'll first do it for the label 'Family'

In [182]:
clf_family = SVC(C=0.01,kernel='rbf',gamma=0.1,decision_function_shape='ovr')
clf_family.fit(X_tr, Y_tr_family)
tr_family_pred=clf_family.predict(X_tr)
print(accuracy_score(Y_tr_family, tr_family_pred))

0.6280778395552026


We see that that for C=10^-3, we get an accuracy as low as 0.6 even for values as high as 100 for gamma. And, we also see that accuracy is greater than 0.7867355043685464 for gamma >= 0.2 (C=0.01). However, it falls below 0.78 for Gamma =0.1 when C=0.01.
We choose our threshold as 78% accuracy for which we get the below ranges for the two parameters.
C=[ 10^-2,10^-1,10^0,10^1,10^2,10^3]
gamma=[0.2,0.3,0.4,.....2]

We'll now perform cross validation accordingly.

In [183]:
import timeit
start_time = timeit.default_timer()
svc_model_family = SVC(kernel='rbf',decision_function_shape='ovr')
params={'C':[0.01,0.1,10**0,10**1,10**2,10**3], 'gamma':[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]}
family_clf = GridSearchCV(svc_model_family, param_grid=params, cv=10)
family_clf.fit(X_tr, Y_tr_family)
elapsed = timeit.default_timer() - start_time
print(elapsed)

240.28467937999994


In [184]:
print("We get the best CV score of:",family_clf.best_score_," when the parameters are:",family_clf.best_params_)

We get the best CV score of: 0.9930500397140588  when the parameters are: {'C': 10, 'gamma': 1.9}


Now we'll use these parameters and train the model and getting our hamming loss and accuracy on test sets.

In [185]:
clf_family_actual = SVC(C=10,kernel='rbf',gamma=1.9,decision_function_shape='ovr')
clf_family_actual.fit(X_tr, Y_tr_family)
ts_family_pred=clf_family_actual.predict(X_ts)
print("The exact match metric/ accuracy score for 'family' label is :", accuracy_score(Y_ts_family, ts_family_pred))
print("The hamming loss for 'family' label is:",hamming_loss(Y_ts_family, ts_family_pred))


The exact match metric/ accuracy score for 'family' label is : 0.9944418712366836
The hamming loss for 'family' label is: 0.00555812876331635


#### We'll now do it for the label 'Genus'

In [186]:
clf_genus = SVC(C=0.01,kernel='rbf',gamma=0.2,decision_function_shape='ovr')
clf_genus.fit(X_tr, Y_tr_genus)
tr_genus_pred=clf_genus.predict(X_tr)
print(accuracy_score(Y_tr_genus, tr_genus_pred))

0.6503177124702144


After trying out different combinations for gamma and C , we see that for C=0.001 gives a lower accuracy than 0.6, even for gamma values as large as 1000. So we chose lower bound of C to be 0.01 and upper bound as 10^3. Also,for C=0.01, we see that we get accuracy lower than 0.76 (choosing this as the threshold for this case) for gamma = 0.1 and 0.2. Hence, our ranges for C =[ 10^-2,10^-1,10^0,10^1,10^2,10^3] and gamma = [0.3,0.4,.....2].

In [187]:
import timeit
start_time = timeit.default_timer()
svc_model_genus = SVC(kernel='rbf',decision_function_shape='ovr')
params={'C':[0.01,0.1,10**0,10**1,10**2,10**3], 'gamma':[0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]}
genus_clf = GridSearchCV(svc_model_genus, param_grid=params, cv=10)
genus_clf.fit(X_tr, Y_tr_genus)
elapsed = timeit.default_timer() - start_time
print(elapsed)

243.30991744399944


In [188]:
print("We get the best CV score of:",genus_clf.best_score_," when the parameters are:",genus_clf.best_params_)

We get the best CV score of: 0.9912629070691025  when the parameters are: {'C': 100, 'gamma': 0.8}


Now we'll use these parameters and train the model and getting our hamming loss and accuracy on test sets.

In [189]:
clf_genus_actual = SVC(C=100,kernel='rbf',gamma=0.8,decision_function_shape='ovr')
clf_genus_actual.fit(X_tr, Y_tr_genus)
ts_genus_pred=clf_genus_actual.predict(X_ts)
print("The exact match metric/ accuracy score for 'genus' label is :", accuracy_score(Y_ts_genus, ts_genus_pred))
print("The hamming loss for 'genus' label is:",hamming_loss(Y_ts_genus, ts_genus_pred))

The exact match metric/ accuracy score for 'genus' label is : 0.9893469198703103
The hamming loss for 'genus' label is: 0.010653080129689671


#### We'll now do it for the label 'Species'

In [190]:
clf_species = SVC(C=0.01,kernel='rbf',gamma=0.3,decision_function_shape='ovr')
clf_species.fit(X_tr, Y_tr_species)
tr_species_pred=clf_species.predict(X_tr)
print(accuracy_score(Y_tr_species, tr_species_pred))

0.6938046068308181


After trying out different combinations for gamma and C , we see that for C=0.001 gives a lower accuracy than 0.5, even for gamma values as large as 1000. So we chose lower bound of C to be 0.01 and upper bound as 10^3. Also,for C=0.01, we see that we get accuracy lower than 0.76 (choosing this as the threshold for this case) for gamma = 0.1,0.2 and 0.3. Hence, our ranges for C =[ 10^-2,10^-1,10^0,10^1,10^2,10^3] and gamma = [0.4,.....2].

In [191]:
import timeit
start_time = timeit.default_timer()
svc_model_species = SVC(kernel='rbf',decision_function_shape='ovr')
params={'C':[0.01,0.1,10**0,10**1,10**2,10**3], 'gamma':[0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]}
species_clf = GridSearchCV(svc_model_species, param_grid=params, cv=10)
species_clf.fit(X_tr, Y_tr_species)
elapsed = timeit.default_timer() - start_time
print(elapsed)

246.53114328899937


In [192]:
print("We get the best CV score of:",species_clf.best_score_," when the parameters are:",species_clf.best_params_)

We get the best CV score of: 0.9908657664813344  when the parameters are: {'C': 10, 'gamma': 1.7}


In [193]:
clf_species_actual = SVC(C=10,kernel='rbf',gamma=1.7,decision_function_shape='ovr')
clf_species_actual.fit(X_tr, Y_tr_species)
ts_species_pred=clf_species_actual.predict(X_ts)
print("The exact match metric/ accuracy score for 'species' label is :", accuracy_score(Y_ts_species, ts_species_pred))
print("The hamming loss for 'species' label is:",hamming_loss(Y_ts_species, ts_species_pred))


The exact match metric/ accuracy score for 'species' label is : 0.9911996294580825
The hamming loss for 'species' label is: 0.008800370541917554


In [318]:
l1=ts_species_pred.tolist()
l2=ts_genus_pred.tolist()
l3=ts_family_pred.tolist()
Y_fcalls_pred=pd.DataFrame({"Family":l3,"Genus":l2,"Species":l1})
s1=np.sum(np.not_equal(Y_ts,Y_fcalls_pred))/float(Y_ts.size)
print("The hamming loss for the classifier is:",1-sum(s1))

counter=0
for x in range(0,len(Y_ts)):
    if (Y_ts.iloc[x].values == Y_fcalls_pred.iloc[x].values).all() == True:
        counter=counter+1
        
print("The exact match score for the classifier is:",counter/len(Y_ts))
    



The hamming loss for the classifier is: 0.9916628068550255
The exact match score for the classifier is: 0.9865678554886521


### Solution for 1(b)(iii) as below:

We'll first standardize the attributes

In [194]:
stdz = preprocessing.StandardScaler()
X_tr_std = stdz.fit_transform(X_tr)
X_ts_std = stdz.fit_transform(X_ts)
X_tr_stddf = pd.DataFrame(X_tr_std,columns=X_tr.columns)
X_ts_stddf = pd.DataFrame(X_ts_std,columns=X_ts.columns)

For this question, we'll directly use cross validation to find the weight of the L1 penalty.
We'll then train our data on the chosen C and find the hamming loss and exact match for each label.

Towards the end, we compute the hamming score and the exact match score for the classifier as a whole as well.

#### We'll now do it for the label 'Family'


In [221]:
import timeit
start_time = timeit.default_timer()
svc_model_familyl1 = LinearSVC(penalty='l1', multi_class='ovr',dual=False)
params={'C':[0.001,0.01,0.1,10**0,10**1,10**2,10**3]}
familyl1_clf = GridSearchCV(svc_model_familyl1, param_grid=params, cv=10)
familyl1_clf.fit(X_tr_stddf, Y_tr_family)
elapsed = timeit.default_timer() - start_time
print(elapsed)

71.55602290599927


In [222]:
print("We get the best CV score of:",familyl1_clf.best_score_," when the weight of the penalty is:",familyl1_clf.best_params_)

We get the best CV score of: 0.938046068308181  when the weight of the penalty is: {'C': 100}


We'll now train the model using the above value of C and check the hamming loss and exact match score on the test set.

In [223]:
clf_familyl1_actual = LinearSVC(penalty='l1', C=100, multi_class='ovr',dual=False)
clf_familyl1_actual.fit(X_tr_stddf, Y_tr_family)
ts_familyl1_pred=clf_familyl1_actual.predict(X_ts_stddf)
print("The exact match metric/ accuracy score for 'family' label is :", accuracy_score(Y_ts_family, ts_familyl1_pred))
print("The hamming loss for 'family' label is:",hamming_loss(Y_ts_family, ts_familyl1_pred))



The exact match metric/ accuracy score for 'family' label is : 0.9328392774432608
The hamming loss for 'family' label is: 0.06716072255673923


#### We'll now do it for the label 'Genus'



In [198]:
import timeit
start_time = timeit.default_timer()
svc_model_genusl1 = LinearSVC(penalty='l1', multi_class='ovr',dual=False)
params={'C':[0.001,0.01,0.1,10**0,10**1,10**2,10**3]}
genusl1_clf = GridSearchCV(svc_model_genusl1, param_grid=params, cv=10)
genusl1_clf.fit(X_tr_stddf, Y_tr_genus)
elapsed = timeit.default_timer() - start_time
print(elapsed)

99.44274292400041


In [199]:
print("We get the best CV score of:",genusl1_clf.best_score_," when the weight of the penalty is:",genusl1_clf.best_params_)


We get the best CV score of: 0.9529388403494837  when the weight of the penalty is: {'C': 1}


We'll now train the model using the above value of C and check the hamming loss and exact match score on the test set.

In [200]:
clf_genusl1_actual = LinearSVC(penalty='l1', C=1, multi_class='ovr',dual=False)
clf_genusl1_actual.fit(X_tr_stddf, Y_tr_genus)
ts_genusl1_pred=clf_genusl1_actual.predict(X_ts_stddf)
print("The exact match metric/ accuracy score for 'genus' label is :", accuracy_score(Y_ts_genus, ts_genusl1_pred))
print("The hamming loss for 'genus' label is:",hamming_loss(Y_ts_genus, ts_genusl1_pred))


The exact match metric/ accuracy score for 'genus' label is : 0.9467345993515517
The hamming loss for 'genus' label is: 0.053265400648448355


#### We'll now do it for the label 'Species'

In [201]:
import timeit
start_time = timeit.default_timer()
svc_model_speciesl1 = LinearSVC(penalty='l1', multi_class='ovr',dual=False)
params={'C':[0.001,0.01,0.1,10**0,10**1,10**2,10**3]}
speciesl1_clf = GridSearchCV(svc_model_speciesl1, param_grid=params, cv=10)
speciesl1_clf.fit(X_tr_stddf, Y_tr_species)
elapsed = timeit.default_timer() - start_time
print(elapsed)

104.05422022600033


In [202]:
print("We get the best CV score of:",speciesl1_clf.best_score_," when the weight of the penalty is:",speciesl1_clf.best_params_)


We get the best CV score of: 0.9584988085782367  when the weight of the penalty is: {'C': 10}


We'll now train the model using the above value of C and check the hamming loss and exact match score on the test set.

In [220]:
clf_speciesl1_actual = LinearSVC(penalty='l1', C=10, multi_class='ovr',dual=False)
clf_speciesl1_actual.fit(X_tr_stddf, Y_tr_species)
ts_speciesl1_pred=clf_speciesl1_actual.predict(X_ts_stddf)
print("The exact match metric/ accuracy score for 'species' label is :", accuracy_score(Y_ts_species, ts_speciesl1_pred))
print("The hamming loss for 'species' label is:",hamming_loss(Y_ts_species, ts_speciesl1_pred))



The exact match metric/ accuracy score for 'species' label is : 0.9573876794812413
The hamming loss for 'species' label is: 0.042612320518758684


In [319]:
l1=ts_speciesl1_pred.tolist()
l2=ts_genusl1_pred.tolist()
l3=ts_familyl1_pred.tolist()
Y_fcalls_pred=pd.DataFrame({"Family":l3,"Genus":l2,"Species":l1})
s1=np.sum(np.not_equal(Y_ts,Y_fcalls_pred))/float(Y_ts.size)
print("The hamming loss for the classifier is:",1-sum(s1))

counter=0
for x in range(0,len(Y_ts)):
    if (Y_ts.iloc[x].values == Y_fcalls_pred.iloc[x].values).all() == True:
        counter=counter+1
        
print("The exact match score for the classifier is:",counter/len(Y_ts))
    

The hamming loss for the classifier is: 0.9456538520920179
The exact match score for the classifier is: 0.9119962945808244


### Solution to 1(b)(iv) as below:

We'll use SMOTE here to deal with class imbalances in different labels.
For each label, we'll first perform SMOTE on the training set. After this we'll cross validate to find the weight of the penalty. We'll take the value of 'C' that gives the highest cross-validation score, and train the model using this value of C. Henceforth, we'll find the exact match metric /accuracy score and the hamming loss on the test set.

Towards the end, we compute the hamming score and the exact match score for the classifier as a whole as well.



#### We'll now do it for the label 'Family'

In [204]:
sm = SMOTE(random_state=42)
X_tr_stddf_sm, Y_tr_family_sm = sm.fit_sample(X_tr_stddf, Y_tr_family)
X_tr_stddf_sm_df=pd.DataFrame(data=X_tr_stddf_sm,columns=X_tr_stddf.columns)
Y_tr_family_sm_df=pd.DataFrame(data=Y_tr_family_sm,columns=Y_tr_family.columns)

In [205]:
Y_tr_family_sm_df['Family'].value_counts()

Leptodactylidae    3093
Bufonidae          3093
Dendrobatidae      3093
Hylidae            3093
Name: Family, dtype: int64

In [206]:
import timeit
start_time = timeit.default_timer()

svc_model_familysm = LinearSVC(penalty='l1', multi_class='ovr',dual=False)
params={'C':[0.001,0.01,0.1,10**0,10**1,10**2,10**3]}
familysm_clf = GridSearchCV(svc_model_familysm, param_grid=params, cv=10)
familysm_clf.fit(X_tr_stddf_sm_df, Y_tr_family_sm_df)

elapsed = timeit.default_timer() - start_time
print(elapsed)

319.476592650999


In [207]:
print("We get the best CV score of:",familysm_clf.best_score_," when the weight of the penalty is:",familysm_clf.best_params_)


We get the best CV score of: 0.9501293242806337  when the weight of the penalty is: {'C': 100}


In [208]:
clf_familysm_actual = LinearSVC(penalty='l1', C=100, multi_class='ovr',dual=False)
clf_familysm_actual.fit(X_tr_stddf_sm_df, Y_tr_family_sm_df)
ts_familysm_pred=clf_familysm_actual.predict(X_ts_stddf)
print("The exact match metric/ accuracy score for 'family' label is :", accuracy_score(Y_ts_family, ts_familysm_pred))
print("The hamming loss for 'family' label is:",hamming_loss(Y_ts_family, ts_familysm_pred))


The exact match metric/ accuracy score for 'family' label is : 0.9147753589624826
The hamming loss for 'family' label is: 0.08522464103751737


#### We'll now do it for the label 'Genus'



In [209]:
sm = SMOTE(random_state=42)
X_tr_genus_sm, Y_tr_genus_sm = sm.fit_sample(X_tr_stddf, Y_tr_genus)
X_tr_genus_sm_df=pd.DataFrame(data=X_tr_genus_sm,columns=X_tr_stddf.columns)
Y_tr_genus_sm_df=pd.DataFrame(data=Y_tr_genus_sm,columns=Y_tr_genus.columns)

In [210]:
Y_tr_genus_sm_df['Genus'].value_counts()

Adenomera        2910
Ameerega         2910
Dendropsophus    2910
Rhinella         2910
Hypsiboas        2910
Scinax           2910
Osteocephalus    2910
Leptodactylus    2910
Name: Genus, dtype: int64

In [211]:
import timeit
start_time = timeit.default_timer()

svc_model_genussm = LinearSVC(penalty='l1', multi_class='ovr',dual=False)
params={'C':[0.001,0.01,0.1,10**0,10**1,10**2,10**3]}
genussm_clf = GridSearchCV(svc_model_genussm, param_grid=params, cv=10)
genussm_clf.fit(X_tr_genus_sm_df, Y_tr_genus_sm_df)

elapsed = timeit.default_timer() - start_time
print(elapsed)

877.7827479900006


In [212]:
print("We get the best CV score of:",genussm_clf.best_score_," when the weight of the penalty is:",genussm_clf.best_params_)


We get the best CV score of: 0.9641752577319588  when the weight of the penalty is: {'C': 10}


In [316]:
clf_genussm_actual = LinearSVC(penalty='l1', C=10, multi_class='ovr',dual=False)
clf_genussm_actual.fit(X_tr_genus_sm_df, Y_tr_genus_sm_df)
ts_genussm_pred=clf_genussm_actual.predict(X_ts_stddf)
print("The exact match metric/ accuracy score for 'genus' label is :", accuracy_score(Y_ts_genus, ts_genussm_pred))
print("The hamming loss for 'genus' label is:",hamming_loss(Y_ts_genus, ts_genussm_pred))



The exact match metric/ accuracy score for 'genus' label is : 0.9027327466419639
The hamming loss for 'genus' label is: 0.09726725335803613


#### We'll now do it for the label 'Species'




In [214]:
sm = SMOTE(random_state=42)
X_tr_species_sm, Y_tr_species_sm = sm.fit_sample(X_tr_stddf, Y_tr_species)
X_tr_species_sm_df=pd.DataFrame(data=X_tr_species_sm,columns=X_tr_stddf.columns)
Y_tr_species_sm_df=pd.DataFrame(data=Y_tr_species_sm,columns=Y_tr_species.columns)

In [215]:
Y_tr_species_sm_df['Species'].value_counts()

HypsiboasCordobae         2449
AdenomeraAndre            2449
AdenomeraHylaedactylus    2449
ScinaxRuber               2449
OsteocephalusOophagus     2449
Rhinellagranulosa         2449
HypsiboasCinerascens      2449
Ameeregatrivittata        2449
HylaMinuta                2449
LeptodactylusFuscus       2449
Name: Species, dtype: int64

In [216]:
import timeit
start_time = timeit.default_timer()

svc_model_speciessm = LinearSVC(penalty='l1', multi_class='ovr',dual=False)
params={'C':[0.001,0.01,0.1,10**0,10**1,10**2,10**3]}
speciessm_clf = GridSearchCV(svc_model_speciessm, param_grid=params, cv=10)
speciessm_clf.fit(X_tr_species_sm_df, Y_tr_species_sm_df)

elapsed = timeit.default_timer() - start_time
print(elapsed)

1091.3786271360004


In [218]:
print("We get the best CV score of:",speciessm_clf.best_score_," when the weight of the penalty is:",speciessm_clf.best_params_)



We get the best CV score of: 0.9653736218864842  when the weight of the penalty is: {'C': 100}


In [219]:
clf_speciessm_actual = LinearSVC(penalty='l1', C=100, multi_class='ovr',dual=False)
clf_speciessm_actual.fit(X_tr_species_sm_df, Y_tr_species_sm_df)
ts_speciessm_pred=clf_speciessm_actual.predict(X_ts_stddf)
print("The exact match metric/ accuracy score for 'species' label is :", accuracy_score(Y_ts_species, ts_speciessm_pred))
print("The hamming loss for 'species' label is:",hamming_loss(Y_ts_species, ts_speciessm_pred))




The exact match metric/ accuracy score for 'species' label is : 0.952292728114868
The hamming loss for 'species' label is: 0.047707271885132005


In [321]:
l1=ts_speciessm_pred.tolist()
l2=ts_genussm_pred.tolist()
l3=ts_familysm_pred.tolist()
Y_fcalls_pred=pd.DataFrame({"Family":l3,"Genus":l2,"Species":l1})
s1=np.sum(np.not_equal(Y_ts,Y_fcalls_pred))/float(Y_ts.size)
print("The hamming loss for the classifier is:",1-sum(s1))

counter=0
for x in range(0,len(Y_ts)):
    if (Y_ts.iloc[x].values == Y_fcalls_pred.iloc[x].values).all() == True:
        counter=counter+1
        
print("The exact match score for the classifier is:",counter/len(Y_ts))
    

The hamming loss for the classifier is: 0.9232669445731049
The exact match score for the classifier is: 0.8545622973598889


For each of the classifiers in b(ii),b(iii),b(iv), we see that the hamming score > exact match score. This is because of the partial correctness it acknowledges, as mentioned in b(i).

We also see that both hamming score and exact match metric reduce when we use L1 penalized SVM.
Both of them reduce even further when we SMOTE on the dataset before performing L1 penalized SVM. This is because the data is no longer heavily imbalanced and does not always predict the majority class(es), leading to very high scores in both of our scoring parameters.

### Solution to 2(a) as below:

We're using CH Index to choose our optimal K for each of the 50 iterations.

In [107]:
import timeit
start_time = timeit.default_timer()

best_kch = []
for x in range(1,51):
    ch_index=[]
    for k in range(2, 51):
        kmeans_cl = KMeans(init = 'random',n_clusters=k).fit(X_fcalls)
        labels = kmeans_cl.labels_
        ch_index.append((k,metrics.calinski_harabaz_score(X_fcalls, labels)))
    sorted_ch_index = sorted(ch_index,key=itemgetter(1),reverse = True)
    print('The optimal K and its CH_index for iteration No',x ,'is',sorted_ch_index[0])
    best_kch.append(sorted_ch_index[0])
    
elapsed = timeit.default_timer() - start_time
print(elapsed)  

The optimal K and its CH_index for iteration No 1 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 2 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 3 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 4 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 5 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 6 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 7 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 8 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 9 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 10 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 11 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 12 is (2, 3680.7413494630937)
The optimal K and its CH_index for iteration No 13 is (2, 3680.7413494630

### Solution to 2(b) as below:

We find the majority label triplet for each cluster in each iteration.

In [77]:
import timeit
start_time = timeit.default_timer()

best_kch = []
for x in range(1,51):
    ch_index=[]
    for k in range(2, 51):
        kmeans_cl = KMeans(init = 'random',n_clusters=k).fit(X_fcalls)
        labels = kmeans_cl.labels_
        ch_index.append((k,labels,metrics.calinski_harabaz_score(X_fcalls, labels)))
        
    sorted_ch_index = sorted(ch_index,key=itemgetter(2),reverse = True)
    print('The optimal K for iteration No',x ,'is',sorted_ch_index[0][0],"and its CH Index is:",sorted_ch_index[0][2])
    print("The clusters are:",np.unique(sorted_ch_index[0][1]))
    array_lab=sorted_ch_index[0][1]
    l=array_lab.tolist()
    cl0=[]
    cl1=[]

    for i in range(0,len(l)):
        if l[i]==0:
            cl0.append(i)
        elif l[i]==1:
            cl1.append(i)
    clus0=[]
    clus1=[]

    fam0=list(Y_fcalls['Family'].iloc[cl0])
    genus0=list(Y_fcalls['Genus'].iloc[cl0])
    species0=list(Y_fcalls['Species'].iloc[cl0])
    
    fam1=list(Y_fcalls['Family'].iloc[cl1])
    genus1=list(Y_fcalls['Genus'].iloc[cl1])
    species1=list(Y_fcalls['Species'].iloc[cl1])
   
    clus0.append(max(set(fam0), key = fam0.count))
    clus0.append(max(set(genus0), key = genus0.count))
    clus0.append(max(set(species0), key = species0.count))
    print("The majority labels for cluster 0 are:",clus0)
    
    clus1.append(max(set(fam1), key = fam1.count))
    clus1.append(max(set(genus1), key = genus1.count))
    clus1.append(max(set(species1), key = species1.count))
    print("The majority labels for cluster 1 are:",clus1)
    print('\n')

elapsed = timeit.default_timer() - start_time
print(elapsed) 
   

The optimal K for iteration No 1 is 2 and its CH Index is: 3680.7413494630937
The clusters are: [0 1]
The majority labels for cluster 0 are: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus']
The majority labels for cluster 1 are: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']


The optimal K for iteration No 2 is 2 and its CH Index is: 3680.7413494630937
The clusters are: [0 1]
The majority labels for cluster 0 are: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus']
The majority labels for cluster 1 are: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']


The optimal K for iteration No 3 is 2 and its CH Index is: 3680.7413494630937
The clusters are: [0 1]
The majority labels for cluster 0 are: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus']
The majority labels for cluster 1 are: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']


The optimal K for iteration No 4 is 2 and its CH Index is: 3680.7413494630937
The clusters are: [0 1]
The majority labels for cluster 0 are

The optimal K for iteration No 30 is 2 and its CH Index is: 3680.7413494630937
The clusters are: [0 1]
The majority labels for cluster 0 are: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']
The majority labels for cluster 1 are: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus']


The optimal K for iteration No 31 is 2 and its CH Index is: 3680.7413494630937
The clusters are: [0 1]
The majority labels for cluster 0 are: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']
The majority labels for cluster 1 are: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus']


The optimal K for iteration No 32 is 2 and its CH Index is: 3680.7413494630937
The clusters are: [0 1]
The majority labels for cluster 0 are: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']
The majority labels for cluster 1 are: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus']


The optimal K for iteration No 33 is 2 and its CH Index is: 3680.7413494630937
The clusters are: [0 1]
The majority labels for cluster 0

### Solution to 2(c) as below:

For each iteration, we provide the hamming distances of each of the labels, and also the hamming loss and the hamming score.

Towards the end, we find the average and standard deviations of these 50 values of hamming losses and hamming scores.

In [307]:
import timeit
start_time = timeit.default_timer()

best_kch = []
hm_loss=[]
hm_score=[]
for x in range(1,51):
    ch_index=[]
    for k in range(2, 51):
        kmeans_cl = KMeans(init = 'random',n_clusters=k).fit(X_fcalls)
        labels = kmeans_cl.labels_
        ch_index.append((k,labels,metrics.calinski_harabaz_score(X_fcalls, labels)))
        
    sorted_ch_index = sorted(ch_index,key=itemgetter(2),reverse = True)
    #print('The optimal K for iteration No',x ,'is',sorted_ch_index[0][0],"and its CH Index is:",sorted_ch_index[0][2])
    #print("The clusters are:",np.unique(sorted_ch_index[0][1]))
    array_lab=sorted_ch_index[0][1]
    l=array_lab.tolist()
    cl0=[]
    cl1=[]

    for i in range(0,len(l)):
        if l[i]==0:
            cl0.append(i)
        elif l[i]==1:
            cl1.append(i)
    clus0=[]
    clus1=[]

    fam0=list(Y_fcalls['Family'].iloc[cl0])
    genus0=list(Y_fcalls['Genus'].iloc[cl0])
    species0=list(Y_fcalls['Species'].iloc[cl0])
    
    fam1=list(Y_fcalls['Family'].iloc[cl1])
    genus1=list(Y_fcalls['Genus'].iloc[cl1])
    species1=list(Y_fcalls['Species'].iloc[cl1])
   
    clus0.append(max(set(fam0), key = fam0.count))
    clus0.append(max(set(genus0), key = genus0.count))
    clus0.append(max(set(species0), key = species0.count))
    
    clus1.append(max(set(fam1), key = fam1.count))
    clus1.append(max(set(genus1), key = genus1.count))
    clus1.append(max(set(species1), key = species1.count))
    
    
    indexl=[]
    faml=[]
    genusl=[]
    speciesl=[]
    
    for j in range(0,len(cl0)):
        indexl.append(cl0[j])
        faml.append(max(set(fam0), key = fam0.count))
        genusl.append(max(set(genus0), key = genus0.count))
        speciesl.append(max(set(species0), key = species0.count))
    for g in range(0,len(cl1)):
        indexl.append(cl1[g])
        faml.append(max(set(fam1), key = fam1.count))
        genusl.append(max(set(genus1), key = genus1.count))
        speciesl.append(max(set(species1), key = species1.count))
        
    comp_df=pd.DataFrame({'Indexes':indexl,'Family':faml,'Genus':genusl,'Species':speciesl})
    comp_df.set_index('Indexes', inplace=True)
    comp_df_sorted=comp_df.sort_index()
    
    hm=np.sum(np.not_equal(Y_fcalls, comp_df_sorted))/float(Y_fcalls.size)
    hd=hm*3
    print("The hamming distances for family,genus and species in iteration",x,"is",hd.values,"respectively")
    print("The hamming loss/avg hamming distance for iteration",x,"is:",sum(hm.values))
    print("The hamming score for iteration",x,"is:",(1-sum(hm.values)))
    print('\n')
    
    hm_loss.append(sum(hm.values))
    hm_score.append((1-sum(hm.values)))
    

elapsed = timeit.default_timer() - start_time
print("time:",elapsed) 
   

The hamming distances for family,genus and species in iteration 1 is [0.23349548 0.29826268 0.36386379] respectively
The hamming loss/avg hamming distance for iteration 1 is: 0.2985406532314107
The hamming score for iteration 1 is: 0.7014593467685892


The hamming distances for family,genus and species in iteration 2 is [0.23349548 0.29826268 0.36386379] respectively
The hamming loss/avg hamming distance for iteration 2 is: 0.2985406532314107
The hamming score for iteration 2 is: 0.7014593467685892


The hamming distances for family,genus and species in iteration 3 is [0.23349548 0.29826268 0.36386379] respectively
The hamming loss/avg hamming distance for iteration 3 is: 0.2985406532314107
The hamming score for iteration 3 is: 0.7014593467685892


The hamming distances for family,genus and species in iteration 4 is [0.23349548 0.29826268 0.36386379] respectively
The hamming loss/avg hamming distance for iteration 4 is: 0.2985406532314107
The hamming score for iteration 4 is: 0.7014593

The hamming distances for family,genus and species in iteration 34 is [0.23349548 0.29826268 0.36386379] respectively
The hamming loss/avg hamming distance for iteration 34 is: 0.2985406532314107
The hamming score for iteration 34 is: 0.7014593467685892


The hamming distances for family,genus and species in iteration 35 is [0.23349548 0.29826268 0.36386379] respectively
The hamming loss/avg hamming distance for iteration 35 is: 0.2985406532314107
The hamming score for iteration 35 is: 0.7014593467685892


The hamming distances for family,genus and species in iteration 36 is [0.23349548 0.29826268 0.36386379] respectively
The hamming loss/avg hamming distance for iteration 36 is: 0.2985406532314107
The hamming score for iteration 36 is: 0.7014593467685892


The hamming distances for family,genus and species in iteration 37 is [0.23349548 0.29826268 0.36386379] respectively
The hamming loss/avg hamming distance for iteration 37 is: 0.2985406532314107
The hamming score for iteration 37 i

In [314]:
print("The average of Hamming Loss over 50 iterations is",np.mean(hm_loss))
print("The average of Hamming Score over 50 iterations is",np.mean(hm_score))


The average of Hamming Loss over 50 iterations is 0.29854065323141077
The average of Hamming Score over 50 iterations is 0.7014593467685891


In [315]:
print("The standard deviation of Hamming Loss over 50 iterations is",np.std(hm_loss))
print("The standard deviation of Hamming Score over 50 iterations is",np.std(hm_score))

The standard deviation of Hamming Loss over 50 iterations is 5.551115123125783e-17
The standard deviation of Hamming Score over 50 iterations is 1.1102230246251565e-16


### Solution to 3: In pdf, in the zip file